In [4]:
!pip install facebook-business



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import sys
print(sys.executable)


C:\Users\HP\Desktop\genesis.ai\env\Scripts\python.exe


In [1]:

import os
import glob
import hashlib
import hmac
from dotenv import load_dotenv
from datetime import datetime, timedelta, timezone

# imports for langchain, plotly and Chroma
from openai import OpenAI

from langchain_community.document_loaders import DirectoryLoader, TextLoader
from langchain.text_splitter import CharacterTextSplitter,RecursiveCharacterTextSplitter
from langchain.schema import Document
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_chroma import Chroma
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
import numpy as np
import plotly.graph_objects as go
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores.utils import filter_complex_metadata
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.tools import tool
from typing import Dict

from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI





from facebook_business.api import FacebookAdsApi
from facebook_business.adobjects.adaccount import AdAccount
from facebook_business.adobjects.adimage import AdImage
from facebook_business.adobjects.adcreative import AdCreative
from facebook_business.adobjects.adset import AdSet
from facebook_business.adobjects.campaign import Campaign
from facebook_business.adobjects.ad import Ad



In [2]:
# Check your available ad accounts
from facebook_business.api import FacebookAdsApi
from facebook_business.adobjects.user import User

try:
    me = User(fbid='me')
    ad_accounts = me.get_ad_accounts()
    
    print("Available Ad Accounts:")
    for account in ad_accounts:
        print(f"ID: {account['id']}, Name: {account['name']}")
        
except Exception as e:
    print(f"Error fetching accounts: {e}")

Error fetching accounts: Api call cannot be made if api is not set


In [3]:

load_dotenv()
db_name = os.getenv("CHROMA_DB_BASE_PATH", "./chroma_db")
persist_dir = db_name  # or another env variable if you want
MODEL = os.getenv("GPT_MODEL", "gpt-4o-mini")

openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')
META_ACCESS_TOKEN =  os.getenv('SYSTEM_ACCESS_TOKEN')
client = OpenAI(api_key=openai_api_key,
                base_url="https://api.openai.com/v1",
                # 🔒 Force new HTTP client to avoid cached proxy
                http_client=None)

# 🔐 Your credentials
ad_account_id = os.getenv('fb_ad_account_id')
app_id = os.getenv('fb_app_id')
app_secret = os.getenv('fb_app_secret')
page_id = os.getenv('fb_page_id')
BASE_URL = "https://graph.facebook.com/v23.0"




In [4]:
# Check token permissions
import requests

url = f"https://graph.facebook.com/me/permissions?access_token={META_ACCESS_TOKEN}"

response = requests.get(url)
permissions = response.json()

print("Current permissions:")
for perm in permissions.get('data', []):
    print(f"- {perm['permission']}: {perm['status']}")

Current permissions:
- read_insights: granted
- pages_manage_cta: granted
- pages_show_list: granted
- ads_management: granted
- ads_read: granted
- business_management: granted
- pages_read_engagement: granted
- pages_read_user_content: granted
- pages_manage_ads: granted
- pages_manage_posts: granted
- pages_manage_engagement: granted
- public_profile: granted


In [182]:
import requests


url = f"https://graph.facebook.com/v19.0/act_{ad_account_id}/campaigns"

payload = {
    "name": "Genesis Test Campaign",
    "objective": "OUTCOME_LEADS",  # or CONVERSIONS, REACH, etc.
    "status": "PAUSED",
    "special_ad_categories":  json.dumps([])
}

headers = {
    "Authorization": f"Bearer {META_ACCESS_TOKEN}"
}

response = requests.post(url, data=payload, headers=headers)
print(response.json())


{'id': '120228651236750511'}


In [183]:
def clone_campaign(meta_campaign_id: str) -> Dict:
    """
    Clones a Meta campaign intelligently using deep copy with suffix timestamp.
    Optionally applies variations based on decision data.

    Args:
        campaign_id (str): Original Meta campaign ID
        decision_data (dict): Optional decision data (for context or variation logic)

    Returns:
        dict: Result of cloning operation
    """
    try:
        # 🎯 Create a unique suffix to identify the clone
        suffix = f"Clone_{datetime.now().strftime('%m%d')}"

        # 🔁 Clone campaign via API
        url = f"{BASE_URL}/{meta_campaign_id}/copies"
        payload = {
            "access_token": META_ACCESS_TOKEN,
            "deep_copy": True,
            "rename_options": {
                "rename_strategy": "APPEND_SUFFIX",
                "rename_suffix": suffix
            },
            "status_option": "INHERITED"  # Start with same status as original
        }

        response = requests.post(url, json=payload)
        response.raise_for_status()
        clone_data = response.json()

        # ✅ Extract new campaign ID from Meta response
        new_campaign_id = clone_data.get("copied_campaign_id")

        # 🛠 Optional: Apply variations (creative, audience, etc.)
        variation_result = apply_clone_variations(new_campaign_id, decision_data)

        return {
            "status": "success",
            "action": "clone",
            "original_meta_campaign_idd": meta_campaign_id,
            "new_meta_campaign_id": new_meta_campaign_id,
            "variations_applied": variation_result,
            "response": clone_data
        }

    except Exception as e:
        return {
            "status": "failed",
            "action": "clone",
            "campaign_id": meta_campaign_id,
            "error": str(e),
            "fallback": "Manual campaign duplication recommended"
        }


In [46]:
clone_campaign('120228457965020511')

{'status': 'failed',
 'action': 'clone',
 'campaign_id': '120228457965020511',
 'error': '400 Client Error: Bad Request for url: https://graph.facebook.com/v23.0/120228457965020511/copies',
 'fallback': 'Manual campaign duplication recommended'}

In [6]:
import requests

API_VERSION = 'v18.0'
BASE_URL = f'https://graph.facebook.com/{API_VERSION}'

def fetch_campaign_metrics(campaign_id: str) -> dict:
    url = f"{BASE_URL}/{campaign_id}/insights"
    params = {
        'access_token': META_ACCESS_TOKEN,
        'fields': ','.join([
            'campaign_name',
            'impressions',
            'reach',
            'frequency',
            'clicks',
            'ctr',
            'cpc',
            'spend',
            'inline_link_clicks',
            'actions',
            'purchase_roas'
        ]),
        'date_preset': 'today'  # adjust timeframe if needed
    }

    resp = requests.get(url, params=params)

    if resp.status_code != 200:
        print("❌ Meta API Error:")
        print(resp.json())  # 👈 this shows the real reason (e.g. permissions, ID error)
        return {"error": resp.json()}

    data = resp.json().get('data', [])
    if not data:
        return {"error": "No data returned from Meta."}
    row = data[0]

    purchases = 0
    for act in row.get('actions', []):
        if act.get('action_type') == 'purchase':
            purchases = int(act.get('value', 0))
            break

    return {
        "campaign_id": campaign_id,
        "campaign_name": row.get("campaign_name", ""),
        "impressions": int(row.get("impressions", 0)),
        "reach": int(row.get("reach", 0)),
        "frequency": float(row.get("frequency", 0)),
        "clicks": int(row.get("inline_link_clicks", row.get("clicks", 0))),
        "ctr": float(row.get("ctr", 0)),
        "cpc": float(row.get("cpc", 0)),
        "spend": float(row.get("spend", 0)),
        "purchases": purchases,
        "purchase_roas": float(row.get("purchase_roas", [{}])[0].get("value", 0)) if row.get("purchase_roas") else 0.0
    }


In [7]:
fetch_campaign_metrics('120228819230470511')

{'error': 'No data returned from Meta.'}

In [37]:
def pause_campaign(campaign_id: str) -> dict:
    """
    Pauses a Meta campaign immediately via API call.
    """
    url = f"{BASE_URL}/{campaign_id}"
    payload = {
        "status": "PAUSED",
        "access_token": META_ACCESS_TOKEN
    }

    try:
        resp = requests.post(url, data=payload)
        resp.raise_for_status()
        return {
            "action": "pause",
            "status": "executed",
            "campaign_id": campaign_id,
            "response": resp.json()
        }
    except Exception as e:
        return {
            "action": "pause",
            "status": "failed",
            "error": str(e),
            "campaign_id": campaign_id
        }


In [38]:
pause_campaign('120228384165850511')

{'action': 'pause',
 'status': 'executed',
 'campaign_id': '120228384165850511',
 'response': {'success': True}}

In [25]:
def get_campaign_status(campaign_id: str) -> dict:
    url = f"{BASE_URL}/{campaign_id}"
    params = {
        'access_token': META_ACCESS_TOKEN,
        'fields': 'status,effective_status,configured_status'
    }

    resp = requests.get(url, params=params)
    return resp.json()


In [26]:
get_campaign_status('120228384165850511')

{'status': 'PAUSED',
 'effective_status': 'PAUSED',
 'configured_status': 'PAUSED',
 'id': '120228384165850511'}

In [172]:
payload = {
    'campaign_name': 'Home Workout routine - Instagram',
    'headline': 'Get Fit at Home—No Equipment Needed!',
    'platform': 'Instagram',
    'objective': 'OUTCOME_TRAFFIC',
    'ad_type': 'Image',  # changed here
    'budget': '30.0',
    'audience': {
        'interests': ['Home Fitness', 'Healthy Living', 'Workout Routines', 'Wellness', 'Time Management'],
        'age_range': '18-45',
        'gender': 'all'
    },
    'creatives': [{
        'type': 'Image',  # changed here
        'url': 'http://res.cloudinary.com/dtlz2vhof/image/upload/v1750464033/e93pxwmumje21fou5a1q.jpg',
        'headline': 'Get Fit at Home—No Equipment Needed!',
        'cta': 'Start Your Fitness Journey Today!',
        'ad_copy': 'Transform your living room into a personal gym! Discover the Ultimate Home Workout Routine designed for busy individuals—get fit without the hassle.'
    }],
    'creatives_attached': 1
}


In [72]:
def reach_estimate(targeting: dict) -> dict:
    """Estimates audience size for given targeting spec (optional)."""
    url = f"https://graph.facebook.com/v19.0/act_{ad_account_id}/reachestimate"
    response = requests.post(url, headers=HEADERS, data={"targeting_spec": json.dumps(targeting)})
    return response.json()


In [267]:
import os
import requests

def fetch_interests(interests, limit=50):
    META_ACCESS_TOKEN = os.getenv('SYSTEM_ACCESS_TOKEN')
    results = []
    
    for interest in interests:
        r = requests.get(
            "https://graph.facebook.com/v19.0/search",
            params={
                "q": interest,
                "type": "adinterest",
                'limit': limit,
                "access_token":META_ACCESS_TOKEN
            }
        )
        data = r.json().get("data", [])
        if data:
            interest_data = data[0]  # Use the first match
            results.append({
                "id": interest_data.get("id"),
                "name": interest_data.get("name"),
                "audience_lower": interest_data.get("audience_size_lower_bound", "N/A"),
                "audience_upper": interest_data.get("audience_size_upper_bound", "N/A")
            })
    
    return results


In [ ]:
fetch_interests([ "Personal trainer",
      "Fitness and wellness",
      "Gym",
      "Health coach",
      "Online coaching",
      "Canva",
      "Graphic design",
      "Content marketing",
      "Entrepreneurship",
      "Instagram",
      "Facebook advertising",
      "Artificial intelligence",
      "Fitness motivation",
      "Workout routines",
      "Small business"])

In [268]:
fetch_interests(['fitness routine'])

[]

In [217]:
import os
import requests

def fetch_behaviors(limit=25):
    """
    Fetch Meta Ads targeting behavior IDs from the Graph API based on behavior names.

    Args:
        behaviors (list[str]): List of behavior keywords to search for.
        limit (int): Max results to return per behavior search (default: 25).

    Returns:
        List[dict]: Each dict contains id, name, and audience size bounds.
    """
    META_ACCESS_TOKEN = os.getenv('SYSTEM_ACCESS_TOKEN')
    ad_account_id = os.getenv("fb_ad_account_id")  # Format: act_1234567890

    url = f"https://graph.facebook.com/v19.0/{ad_account_id}/targetingcategories"
    params = {
        "class": "behaviors",
        "access_token": access_token,
        "limit": 1000
    }

    r = requests.get(url, params=params)
    print("🔎 Behavior Fetch Response:", r.json())  # Debug

    return r.json().get("data", [])


In [210]:
behaviors_to_fetch = fitness_wellness_behaviors = [
    "Online buyers",
    "Frequent online buyers",
    "Engaged shoppers",
    "Technology early adopters",
    "Small business owners",
    "Health and wellness",
    "Fitness enthusiasts",
    "Self-employed",
    "Frequent social media users",
    "Digital activities",
    "Purchase behavior",
    "Business marketing",
    "Online education",
    "E-commerce shoppers",
    "Canva users"  # may not match directly, just for test
]


behavior_objs = fetch_behaviors(behaviors_to_fetch)
print(behavior_objs)

[]


In [218]:
behaviors = ["online shopping", "frequent travelers", ""]
fetch_behaviors()

🔎 Behavior Fetch Response: {'error': {'message': 'Unknown path components: /targetingcategories', 'type': 'OAuthException', 'code': 2500, 'fbtrace_id': 'A3i8QcUDxGdSILRKWAzYy7U'}}


[]

In [201]:
campaign = {
'campaign_name': 'Home Workout routine - Instagram',
'campaign_id': 8,
'headline': 'Get Fit at Home - No Gym Needed!',
'platform': 'Instagram',
'objective': 'conversions',
'ad_type': 'Video',
'budget': '20.00',
 'audience': {
             'interests': ['Fitness', 'Home workouts', 'Health and wellness', 'Yoga', 'Bodyweight training'], 
              'behaviors': ['Frequent gym-goers looking for alternatives', 'Busy professionals seeking fitness solutions', 'Health-conscious individuals'],
              'demographics': {'age_min': 20, 'age_max': 40,
              'education_statuses': ['High School', 'College grads'],
                'relationship_statuses': ['Single', 'Married', 'In a relationship']},
              'gender': 'all'
             },
'creatives': [{'type': 'Video', 'url': 'http://res.cloudinary.com/dtlz2vhof/image/upload/v1750464033/e93pxwmumje21fou5a1q.jpg',
'headline': 'Get Fit at Home - No Gym Needed!',
'cta': 'Get Started', 'ad_copy': 'Transform your home into a personal gym with our Ultimate Home Workout Routine. No equipment required, just your determination!'}],
'creatives_attached': 1,
'start_date': (datetime.now(timezone.utc) + timedelta(minutes=5)).isoformat(),
'end_time': None

}

In [202]:


def create_campaign_on_meta(campaign):
    """
    Creates a campaign on Meta's ad platform using the given campaign dict.
    Also prepares ad set payload with targeting details (interest, gender, age, geo, behaviors).
    Returns the ad set payload and the created meta campaign ID.
    """
    headers = {
        "Authorization": f"Bearer {META_ACCESS_TOKEN}"
    }

    # === 1. Create Campaign ===
    campaign_payload = {
        "name": campaign["headline"],
        "objective": campaign["objective"],
        "status": "PAUSED",
        "special_ad_categories": json.dumps([]),
    }

    campaign_url = f"{BASE_URL}/act_{AD_ACCOUNT_ID}/campaigns"
    campaign_response = requests.post(campaign_url, data=campaign_payload, headers=headers)
    campaign_data = campaign_response.json()
    print("🧱 Campaign Response:", campaign_data)

    if 'id' not in campaign_data:
        return {"error": "Campaign creation failed", "details": campaign_data}

    meta_campaign_id = campaign_data['id']

    # === 2. Fetch Interest IDs ===
    raw_interests = campaign["audience"].get("interests", [])
    interest_objs = fetch_interests(raw_interests)

    # === 3. Create Ad Set Payload ===
    targeting = {
        "age_min": campaign["audience"]["demographics"]["age_min"],
        "age_max": campaign["audience"]["demographics"]["age_max"],
        "genders": [1] if campaign["audience"]["gender"] == 'male' else [2] if campaign["audience"]["gender"] == 'female' else [],
        "geo_locations": {
            "countries": ["US"],
            "location_types": ["home", "recent"]
        },
        "interests": interest_objs,
        # You can dynamically fetch behavior IDs too and include them here
        "targeting_automation": {
        "advantage_audience": 1  # explicitly required
    }
    }

    adset_payload = {
        "name": f"AdSet for {campaign['headline']}",
        "campaign_id": meta_campaign_id,
        "daily_budget": int(float(campaign["budget"]) * 100),
        "billing_event": "IMPRESSIONS",
        "optimization_goal": "REACH",
        "bid_strategy": "LOWEST_COST_WITHOUT_CAP",
        "start_time": campaign["start_date"],

        "creatives": campaign['creatives'][0]["url"],
        "end_time": campaign["end_time"] if campaign["end_time"] else (datetime.now(timezone.utc) + timedelta(days=5)).isoformat(),
        "targeting": targeting,
        "status": "PAUSED"
    }

    return adset_payload, meta_campaign_id


In [203]:
campaign = {
'campaign_name': 'Home Workout routine - Instagram',
'campaign_id': 8,
'headline': 'Get Fit at Home - No Gym Needed!',
'platform': 'Instagram',
'objective': 'OUTCOME_TRAFFIC',
'ad_type': 'Video',
'budget': '20.00',
 'audience': {
             'interests': ['Fitness', 'Home workouts', 'Health and wellness', 'Yoga', 'Bodyweight training'], 
              'behaviors': ['Frequent gym-goers looking for alternatives', 'Busy professionals seeking fitness solutions', 'Health-conscious individuals'],
              'demographics': {'age_min': 20, 'age_max': 40,
              'education_statuses': ['High School', 'College grads'],
                'relationship_statuses': ['Single', 'Married', 'In a relationship']},
              'gender': 'all'
             },
'creatives': [{'type': 'Video', 'url': 'http://res.cloudinary.com/dtlz2vhof/image/upload/v1750464033/e93pxwmumje21fou5a1q.jpg',
'headline': 'Get Fit at Home - No Gym Needed!',
'cta': 'Get Started', 'ad_copy': 'Transform your home into a personal gym with our Ultimate Home Workout Routine. No equipment required, just your determination!'}],
'creatives_attached': 1,
'start_date': (datetime.now(timezone.utc) + timedelta(minutes=5)).isoformat(),
'end_time': None

}
campaign = create_campaign_on_meta(campaign)
print(campaign)

🧱 Campaign Response: {'id': '120228240603530511'}
EAAY9yE00JRABOzdoOdNUwBnObpq6LZBemFh2k3zMyoUKPu5aF0owpT4P0DfazmC6z462lvWfAjpFCzBZBnun0DmGDuEPrMPwZCCKADa6HPrjy5mgqDFwq7Q54xJutlv4srCijrgs2qZA6BpUz8luwrrNKPn3QSrxHAQvWAcJfy1BXixWDgssNCGazqmGvBujvgZDZD
({'name': 'AdSet for Get Fit at Home - No Gym Needed!', 'campaign_id': '120228240603530511', 'daily_budget': 2000, 'billing_event': 'IMPRESSIONS', 'optimization_goal': 'REACH', 'bid_strategy': 'LOWEST_COST_WITHOUT_CAP', 'start_time': '2025-06-22T21:20:32.322877+00:00', 'creatives': 'http://res.cloudinary.com/dtlz2vhof/image/upload/v1750464033/e93pxwmumje21fou5a1q.jpg', 'end_time': '2025-06-27T21:15:37.383105+00:00', 'targeting': {'age_min': 20, 'age_max': 40, 'genders': [], 'geo_locations': {'countries': ['US'], 'location_types': ['home', 'recent']}, 'interests': [{'id': '6003384248805', 'name': 'Fitness and wellness (fitness)'}, {'id': '6899293586673', 'name': 'Fitness services (fitness)'}, {'id': '6003246823885', 'name': 'Health and Welln

In [204]:
adset_payload = {'name': 'AdSet for Get Fit at Home - No Gym Needed!', 'campaign_id': '120228240603530511', 'daily_budget': 2000, 'billing_event': 'IMPRESSIONS', 'optimization_goal': 'REACH', 'bid_strategy': 'LOWEST_COST_WITHOUT_CAP', 'start_time': '2025-06-22T21:20:32.322877+00:00', 'creatives': 'http://res.cloudinary.com/dtlz2vhof/image/upload/v1750464033/e93pxwmumje21fou5a1q.jpg', 'end_time': '2025-06-27T21:15:37.383105+00:00', 'targeting': {'age_min': 20, 'age_max': 40, 'genders': [], 'geo_locations': {'countries': ['US'], 'location_types': ['home', 'recent']}, 'interests': [{'id': '6003384248805', 'name': 'Fitness and wellness (fitness)'}, {'id': '6899293586673', 'name': 'Fitness services (fitness)'}, {'id': '6003246823885', 'name': 'Health and Wellness Center (Marshall University)'}, {'id': '6003306084421', 'name': 'Yoga (spirituality)'}], 'targeting_automation': {'advantage_audience': 1}}, 'status': 'PAUSED'}
print(adset_payload)

{'name': 'AdSet for Get Fit at Home - No Gym Needed!', 'campaign_id': '120228240603530511', 'daily_budget': 2000, 'billing_event': 'IMPRESSIONS', 'optimization_goal': 'REACH', 'bid_strategy': 'LOWEST_COST_WITHOUT_CAP', 'start_time': '2025-06-22T21:20:32.322877+00:00', 'creatives': 'http://res.cloudinary.com/dtlz2vhof/image/upload/v1750464033/e93pxwmumje21fou5a1q.jpg', 'end_time': '2025-06-27T21:15:37.383105+00:00', 'targeting': {'age_min': 20, 'age_max': 40, 'genders': [], 'geo_locations': {'countries': ['US'], 'location_types': ['home', 'recent']}, 'interests': [{'id': '6003384248805', 'name': 'Fitness and wellness (fitness)'}, {'id': '6899293586673', 'name': 'Fitness services (fitness)'}, {'id': '6003246823885', 'name': 'Health and Wellness Center (Marshall University)'}, {'id': '6003306084421', 'name': 'Yoga (spirituality)'}], 'targeting_automation': {'advantage_audience': 1}}, 'status': 'PAUSED'}


In [205]:
def ads_set_meta(adset_payload):
    """
    Creates ad set and ad creative using the ad set payload.
    Returns the meta ad set ID and creative ID.
    """
    headers = {
        "Authorization": f"Bearer {META_ACCESS_TOKEN}",
         "Content-Type": "application/json"

    }

    adset_url = f"{BASE_URL}/act_{AD_ACCOUNT_ID}/adsets"
    adset_response = requests.post(adset_url, json=adset_payload, headers=headers)
    adset_data = adset_response.json()
    print("🎯 Ad Set Response:", adset_data)

    if 'id' not in adset_data:
        return {"error": "AdSet creation failed", "details": adset_data}

    meta_adset_id = adset_data['id']
    return meta_adset_id


In [195]:
ads_set_meta(adset_payload)

🎯 Ad Set Response: {'id': '120228239729310511'}


'120228239729310511'

In [218]:
def ad_creative(campaign):
    """
    Creates a Meta Ad Creative using the first creative linked to the campaign.
    Requires the campaign to have at least one creative with a valid image/video link.
    Returns the Meta creative ID if successful.
    """
    headers = {
        "Authorization": f"Bearer {META_ACCESS_TOKEN}",
        "Content-Type": "application/json"
    }

    page_id = os.getenv("page_id")  # ensure this is loaded correctly

    #creative = campaign["creatives"][0]  # assuming it's a list of dicts
    image_url = campaign['creatives'][0]["url"]
    

    # First upload the image to Meta to get its image hash
    image_upload_url = f"{BASE_URL}/act_{AD_ACCOUNT_ID}/adimages"
    image_upload_response = requests.post(
        image_upload_url,
        data={"url": image_url},
        headers={"Authorization": f"Bearer {META_ACCESS_TOKEN}"}
    )
    image_upload_data = image_upload_response.json()
    print("🖼️ Image Upload Response:", image_upload_data)

    if "images" not in image_upload_data:
        return {"error": "Image upload failed", "details": image_upload_data}

    image_hash = list(image_upload_data["images"].values())[0]["hash"]

    creative_payload = {
        "name": f"Creative for {campaign['headline']}",
        "object_story_spec": {
            "page_id": page_id,
            "link_data": {
                "image_hash": image_hash,
                "link": image_url,
                "message": campaign["ad_copy"],
                "name": campaign["headline"],
                "call_to_action": {"type": campaign["cta"].upper()}
            }
        }
    }

    creative_url = f"{BASE_URL}/act_{AD_ACCOUNT_ID}/adcreatives"
    creative_response = requests.post(creative_url, json=creative_payload, headers=headers)
    creative_data = creative_response.json()
    print("🎨 Creative Response:", creative_data)

    if 'id' not in creative_data:
        return {"error": "Creative creation failed", "details": creative_data}

    meta_creative_id = creative_data['id']
    return meta_creative_id


In [219]:
campaign_creative = {
'campaign_name': 'Home Workout routine - Instagram',
'campaign_id': 8,
'headline': 'Get Fit at Home - No Gym Needed!',
'platform': 'Instagram',
'objective': 'OUTCOME_TRAFFIC',
'ad_type': 'Video',
'budget': '20.00',
 'audience': {
             'interests': ['Fitness', 'Home workouts', 'Health and wellness', 'Yoga', 'Bodyweight training'], 
              'behaviors': ['Frequent gym-goers looking for alternatives', 'Busy professionals seeking fitness solutions', 'Health-conscious individuals'],
              'demographics': {'age_min': 20, 'age_max': 40,
              'education_statuses': ['High School', 'College grads'],
                'relationship_statuses': ['Single', 'Married', 'In a relationship']},
              'gender': 'all'
             },
'creatives': [{'type': 'Video', 'url': 'http://res.cloudinary.com/dtlz2vhof/image/upload/v1750464033/e93pxwmumje21fou5a1q.jpg',
'headline': 'Get Fit at Home - No Gym Needed!',
'cta': 'Get Started', 'ad_copy': 'Transform your home into a personal gym with our Ultimate Home Workout Routine. No equipment required, just your determination!'}],
'creatives_attached': 1,
'start_date': (datetime.now(timezone.utc) + timedelta(minutes=5)).isoformat(),
'end_time': None

}

# Step 1: Create Campaign + AdSet

# Step 2: Create AdSet

# Step 3: Create Ad Creative
creative_id = ad_creative(campaign_creative)  # now campaign still refers to the original dict

# ✅ Output results
print("🎨 Creative ID:", creative_id)


🖼️ Image Upload Response: {'error': {'message': '(#3) Application does not have the capability to make this API call.', 'type': 'OAuthException', 'code': 3, 'fbtrace_id': 'AwTGueF5rVsVsGj4s_tsRjF'}}
🎨 Creative ID: {'error': 'Image upload failed', 'details': {'error': {'message': '(#3) Application does not have the capability to make this API call.', 'type': 'OAuthException', 'code': 3, 'fbtrace_id': 'AwTGueF5rVsVsGj4s_tsRjF'}}}


In [ ]:

def create_ad(name="Genesis Ad", ad_set_id=None, creative_id=None):
    url = f"{BASE_URL}/{AD_ACCOUNT_ID}/ads"
    payload = {
        "name": name,
        "adset_id": ad_set_id,
        "creative": json.dumps({"creative_id": creative_id}),
        "status": "PAUSED"
    }
    response = requests.post(url, data=payload, headers=HEADERS)
    return response.json()



In [ ]:

# 🔁 Main Automation Flow

if __name__ == "__main__":
    print("🚀 Starting Genesis Meta Ads Automation...\n")

    # 1. Create Campaign
    campaign = create_campaign()
    campaign_id = campaign.get("id")
    print("✅ Campaign ID:", campaign_id)

    # 2. Create Ad Set
    ad_set = create_ad_set(campaign_id=campaign_id)
    ad_set_id = ad_set.get("id")
    print("✅ Ad Set ID:", ad_set_id)

    # 3. Create Creative
    creative = create_creative(
        image_url="https://cdn.yourdomain.com/images/ad-banner.jpg",
        message="🚀 Boost your business with Genesis AI!",
        link="https://genesis.ai"
    )
    creative_id = creative.get("id")
    print("✅ Creative ID:", creative_id)

    # 4. Create Ad
    ad = create_ad(ad_set_id=ad_set_id, creative_id=creative_id)
    ad_id = ad.get("id")
    print("✅ Ad ID:", ad_id)

    print("\n🎉 Automation completed.")

In [59]:
import random

meta_campaign_id = "123456789012345"
campaign_name = "Test Campaign - Home Workout Gear"
days_running = 6
revenue = 320.00
spend = 200.00
purchases = 8
roas = round(revenue / spend, 2)  # 1.60

metrics = {
    "meta_campaign_id": meta_campaign_id,
    "campaign_name": campaign_name,
    "impressions": 18500,
    "reach": 9400,
    "frequency": round(18500 / 9400, 2),  # 1.97
    "clicks": 290,
    "ctr": round((290 / 18500) * 100, 2),  # ~1.57%
    "cpc": round(spend / 290, 2),  # ~$0.69
    "spend": spend,
    "purchases": purchases,
    "purchase_roas": roas,
    "profit": round(revenue - spend, 2),  # $120.00
    "days_running": days_running
}


In [67]:
def analyze_campaign_metrics(metrics: dict) -> dict:
    """
    Analyze Meta campaign metrics with adaptive scoring and smart flagging.
    Provides actionable insights for campaign optimization.
    """

    # ----------------------------
    # ✅ Extract Raw Metrics
    # ----------------------------
    meta_campaign_id = metrics.get('meta_campaign_id')
    ctr = metrics.get("ctr", 0)
    roas = metrics.get("purchase_roas", 0)
    cpc = metrics.get("cpc", 0)
    spend = metrics.get("spend", 0)
    conversions = metrics.get("purchases", 0)
    frequency = metrics.get("frequency", 0)

    # Additional metrics for better analysis
    impressions = metrics.get("impressions", 0)
    clicks = metrics.get("clicks", 0)
    cpm = metrics.get("cpm", 0)
    campaign_objective = metrics.get("objective", "conversions")  # Default to conversions
    days_running = metrics.get("days_running", 1)  # How long campaign has been active

    # ----------------------------
    # ✅ Calculate Derived Metrics
    # ----------------------------
    # Proper conversion rate calculation
    conversion_rate = (conversions / clicks * 100) if clicks > 0 else 0

    # Cost per conversion
    cost_per_conversion = spend / conversions if conversions > 0 else 0

    # Daily spend rate
    daily_spend = spend / days_running if days_running > 0 else spend

    # Get campaign object (assuming this exists in your system)
    try:
        campaign = agent_models.Campaign.objects.get(meta_campaign_id=meta_campaign_id)
    except:
        campaign = None

    # ----------------------------
    # Initialize Containers
    # ----------------------------
    recommendations = []
    flags = []
    score = 50  # Start with neutral score, not 0

    # ----------------------------
    # 1. CTR Analysis — Creative Effectiveness
    # ----------------------------
    # Industry benchmark: 0.9-1.5% is average, 2%+ is excellent
    if ctr < 0.5:
        flags.append("critically_low_ctr")
        score -= 25
        recommendations.append(
            "🔴 CTR critically low (<0.5%). Creative needs immediate overhaul - test new hooks, visuals, and copy.")
    elif ctr < 0.9:
        flags.append("low_ctr")
        score -= 15
        recommendations.append("🟡 CTR below industry average. Test more engaging creatives and stronger hooks.")
    elif ctr < 1.5:
        score += 5
        recommendations.append("🟢 CTR is decent. Room for improvement with creative optimization.")
    elif ctr < 2.5:
        score += 15
        recommendations.append("🟢 Strong CTR! Your creative is resonating well with the audience.")
    else:
        score += 20
        recommendations.append("🟢 Excellent CTR! Your creative is highly engaging.")

    # Creative fatigue detection
    if ctr < 1.0 and frequency > 2.0:
        flags.append("creative_fatigue")
        recommendations.append("🟡 Low CTR + high frequency indicates creative fatigue. Refresh ads immediately.")

    # ----------------------------
    # 2. ROAS Analysis — Profitability
    # ----------------------------
    # Adjust thresholds based on business model (typically 3:1 minimum for profitability)
    if roas == 0:
        if spend > 15:  # More reasonable threshold
            flags.append("zero_roas_high_spend")
            score -= 30
            recommendations.append("🔴 No ROAS after significant spend. Pause and investigate conversion tracking.")
        else:
            flags.append("zero_roas_early")
            score -= 10
            recommendations.append("🟡 No ROAS yet - normal for new campaigns. Monitor closely.")
    elif roas < 2.0:
        flags.append("unprofitable_roas")
        score -= 20
        recommendations.append("🔴 ROAS below breakeven. Optimize targeting, landing page, or pause campaign.")
    elif roas < 3.0:
        flags.append("marginal_roas")
        score += 0
        recommendations.append("🟡 ROAS near breakeven. Optimize for better profitability.")
    elif roas < 5.0:
        score += 20
        recommendations.append("🟢 Good ROAS! Campaign is profitable.")
    else:
        score += 30
        recommendations.append("🟢 Excellent ROAS! Scale this campaign carefully.")

    # ----------------------------
    # 3. CPC Analysis — Cost Efficiency
    # ----------------------------
    # CPC thresholds vary by industry, but these are more realistic
    if cpc > 2.0:
        flags.append("high_cpc")
        score -= 15
        recommendations.append("🔴 CPC too high. Refine targeting, improve quality score, or test new audiences.")
    elif cpc > 1.0:
        flags.append("moderate_cpc")
        score -= 5
        recommendations.append("🟡 CPC is moderate. Look for optimization opportunities.")
    elif cpc > 0.5:
        score += 5
        recommendations.append("🟢 CPC is reasonable for your market.")
    else:
        score += 10
        recommendations.append("🟢 Excellent CPC! Very cost-efficient traffic.")

    # ----------------------------
    # 4. Frequency Analysis — Ad Fatigue
    # ----------------------------
    if frequency >= 4.0:
        flags.append("severe_ad_fatigue")
        score -= 25
        recommendations.append("🔴 Severe ad fatigue (4+ frequency). Immediately refresh creative or expand audience.")
    elif frequency >= 2.5:
        flags.append("ad_fatigue")
        score -= 15
        recommendations.append("🟡 High frequency detected. Plan creative refresh or audience expansion.")
    elif frequency >= 1.5:
        score += 5
        recommendations.append("🟢 Frequency is optimal - good reach without oversaturation.")
    elif frequency < 1.2:
        if spend > 30:  # Only flag if significant spend
            flags.append("low_frequency_high_spend")
            score -= 5
            recommendations.append("🟡 Low frequency despite high spend. Audience might be too broad.")
        else:
            score += 10
            recommendations.append("🟢 Efficient frequency - reaching fresh audiences.")

    # ----------------------------
    # 5. Conversion Analysis — Funnel Health
    # ----------------------------
    if conversions == 0:
        if spend > 15:
            flags.append("no_conversions_significant_spend")
            score -= 35
            recommendations.append(
                "🔴 No conversions after $30+ spend. Check conversion tracking, landing page, and offer.")
        elif spend > 10:
            flags.append("no_conversions_moderate_spend")
            score -= 20
            recommendations.append("🟡 No conversions yet. Monitor closely and check funnel optimization.")
        else:
            score -= 5
            recommendations.append("🟡 Early stage - allow time for conversion data.")
    else:
        # Analyze conversion rate (clicks to conversions)
        if conversion_rate < 1.0:
            flags.append("low_conversion_rate")
            score -= 15
            recommendations.append("🔴 Low conversion rate (<1%). Optimize landing page, pricing, or offer.")
        elif conversion_rate < 2.0:
            score += 0
            recommendations.append("🟡 Conversion rate needs improvement. Test landing page elements.")
        elif conversion_rate < 5.0:
            score += 10
            recommendations.append("🟢 Good conversion rate. Consider scaling.")
        else:
            score += 20
            recommendations.append("🟢 Excellent conversion rate! Scale with confidence.")

    # ----------------------------
    # 6. Spend Velocity Analysis
    # ----------------------------
    if daily_spend > 20:
        if roas < 3.0:
            flags.append("high_spend_low_return")
            score -= 10
            recommendations.append("🟡 High daily spend with low ROAS. Consider reducing budget until optimized.")
    elif daily_spend < 10:
        if cpc < 0.5:  # If CPC is low but spend is low, might need more budget
            recommendations.append("🟢 Low spend with efficient CPC. Consider increasing budget.")

    # ----------------------------
    # 7. Overall Campaign Health Indicators
    # ----------------------------
    # Good metrics together
    if ctr > 1.5 and roas > 3.0 and conversion_rate > 2.0:
        score += 15
        recommendations.append("🟢 🎯 Campaign is firing on all cylinders! Consider scaling.")
        flags.append("scaling_opportunity")

    # Bad metrics together (compounding issues)
    if ctr < 1.0 and roas < 2.0 and frequency > 2.5:
        score -= 15
        flags.append("multiple_issues")
        recommendations.append("🔴 Multiple performance issues detected. Pause and restructure campaign.")

    # ----------------------------
    # Normalize Score: Clamp 0–100
    # ----------------------------
    score = max(0, min(100, score))

    # ----------------------------
    # Final Status Assignment
    # ----------------------------
    if score < 30:
        status = "critical"
        status_emoji = "🔴"
    elif score < 50:
        status = "underperforming"
        status_emoji = "🟡"
    elif score < 75:
        status = "performing"
        status_emoji = "🟢"
    else:
        status = "excellent"
        status_emoji = "🟢"

    # ----------------------------
    # Priority Action Items
    # ----------------------------
    priority_actions = []

    if "critically_low_ctr" in flags or "severe_ad_fatigue" in flags:
        priority_actions.append("URGENT: Refresh creative immediately")

    if "zero_roas_high_spend" in flags or "no_conversions_significant_spend" in flags:
        priority_actions.append("URGENT: Check conversion tracking and landing page")

    if "scaling_opportunity" in flags:
        priority_actions.append("OPPORTUNITY: Scale budget by 20-30%")

    # ----------------------------
    # Return Comprehensive Analysis
    # ----------------------------
    analysis = {
        "meta_campaign_id": meta_campaign_id,
        "status": status,
        "status_emoji": status_emoji,
        "score": score,
        "metrics": {
            "roas": roas,
            "ctr": ctr,
            "cpc": cpc,
            "conversion_rate": round(conversion_rate, 2),
            "cost_per_conversion": round(cost_per_conversion, 2),
            "frequency": frequency,
            "daily_spend": round(daily_spend, 2)
        },
        "flags": flags,
        "recommendations": recommendations,
        "priority_actions": priority_actions,
        "analysis_summary": f"{status_emoji} Campaign is {status} with {score}/100 health score"
    }
    return analysis

In [68]:
analysis = analyze_campaign_metrics(metrics)

In [69]:
print(analysis)

{'meta_campaign_id': '123456789012345', 'status': 'performing', 'status_emoji': '🟢', 'score': 55, 'metrics': {'roas': 1.6, 'ctr': 1.57, 'cpc': 0.69, 'conversion_rate': 2.76, 'cost_per_conversion': 25.0, 'frequency': 1.97, 'daily_spend': 33.33}, 'flags': ['unprofitable_roas', 'high_spend_low_return'], 'recommendations': ['🟢 Strong CTR! Your creative is resonating well with the audience.', '🔴 ROAS below breakeven. Optimize targeting, landing page, or pause campaign.', '🟢 CPC is reasonable for your market.', '🟢 Frequency is optimal - good reach without oversaturation.', '🟢 Good conversion rate. Consider scaling.', '🟡 High daily spend with low ROAS. Consider reducing budget until optimized.'], 'priority_actions': [], 'analysis_summary': '🟢 Campaign is performing with 55/100 health score'}


In [73]:
def decide_campaign_action(metrics: dict, analysis: dict) -> dict:
    """
    Makes intelligent, profit-first decisions for Meta campaigns based on comprehensive analysis.
    Decisions: pause, wait, scale, clone, edit_creative, change_audience, revise_offer, optimize_budget
    """

    # ----------------------------
    # Extract Core Metrics
    # ----------------------------
    roas = metrics.get("purchase_roas", 0)
    spend = metrics.get("spend", 0)
    conversions = metrics.get("purchases", 0)
    cpc = metrics.get("cpc", 0)
    ctr = metrics.get("ctr", 0)
    frequency = metrics.get("frequency", 0)
    days_running = metrics.get("days_running", 1)
    clicks = metrics.get("clicks", 0)
    impressions = metrics.get("impressions", 0)

    # Analysis outputs
    score = analysis.get("score", 0)
    status = analysis.get("status", "")
    meta_campaign_id = analysis.get("meta_campaign_id", "")
    flags = analysis.get("flags", [])
    recommendations = analysis.get("recommendations", [])
    priority_actions = analysis.get("priority_actions", [])

    # Derived metrics
    daily_spend = spend / days_running if days_running > 0 else spend
    conversion_rate = (conversions / clicks * 100) if clicks > 0 else 0

    # ----------------------------
    # Decision Logic (Priority Order)
    # ----------------------------

    # 🚨 CRITICAL: Immediate pause situations
    if _should_pause_immediately(spend, conversions, roas, flags, days_running):
        return _create_decision("pause",
                                _get_pause_reason(spend, conversions, roas, flags),
                                metrics, analysis, "critical")

    # 🎯 SCALING: Profitable campaigns ready to grow
    if _should_scale(roas, score, conversions, flags, spend, days_running):
        return _create_decision("scale",
                                _get_scale_reason(roas, score, conversions),
                                metrics, analysis, "high")

    # 🧬 CLONING: Excellent performers for horizontal scaling
    if _should_clone(status, roas, conversions, score, flags):
        return _create_decision("clone",
                                _get_clone_reason(roas, conversions, score),
                                metrics, analysis, "high")

    # 🔄 CREATIVE REFRESH: Creative issues detected
    if _needs_creative_refresh(flags, ctr, frequency, conversions, clicks):
        return _create_decision("edit_creative",
                                _get_creative_reason(flags, ctr, frequency),
                                metrics, analysis, "medium")

    # 🎯 AUDIENCE OPTIMIZATION: Targeting issues
    if _needs_audience_change(flags, cpc, conversion_rate, roas):
        return _create_decision("change_audience",
                                _get_audience_reason(flags, cpc, conversion_rate),
                                metrics, analysis, "medium")

    # 🛠 OFFER OPTIMIZATION: Funnel issues
    if _needs_offer_revision(flags, ctr, conversion_rate, conversions, clicks):
        return _create_decision("revise_offer",
                                _get_offer_reason(flags, conversion_rate, ctr),
                                metrics, analysis, "medium")

    # 💰 BUDGET OPTIMIZATION: Spend efficiency issues
    if _needs_budget_optimization(daily_spend, roas, cpc, score):
        return _create_decision("optimize_budget",
                                _get_budget_reason(daily_spend, roas, cpc),
                                metrics, analysis, "low")

    # ⏳ WAIT: Early stage or insufficient data
    if _should_wait(spend, days_running, conversions, impressions):
        return _create_decision("wait",
                                _get_wait_reason(spend, days_running, conversions),
                                metrics, analysis, "low")

    # 🔧 DEFAULT: General optimization needed
    return _create_decision("edit_creative",
                            "Campaign performance needs improvement. Start with creative optimization.",
                            metrics, analysis, "medium")


# ----------------------------
# Decision Logic Functions
# ----------------------------

def _should_pause_immediately(spend, conversions, roas, flags, days_running):
    """Determine if campaign should be paused immediately"""
    # High spend, no conversions, sufficient time
    if spend >= 15 and conversions == 0 and days_running >= 3:
        return True

    # Very high spend, terrible ROAS
    if spend >= 30 and roas < 0.5:
        return True

    # Critical flags
    critical_flags = ["zero_roas_high_spend", "no_conversions_significant_spend",
                      "multiple_issues", "critically_low_ctr"]
    if any(flag in flags for flag in critical_flags):
        return True

    return False


def _should_scale(roas, score, conversions, flags, spend, days_running):
    """Determine if campaign is ready for scaling"""
    # Strong performance indicators
    if (roas >= 4.0 and score >= 80 and conversions >= 5 and
            "scaling_opportunity" in flags and spend >= 30):
        return True

    # Good performance with proven data
    if (roas >= 3.5 and score >= 75 and conversions >= 3 and
            days_running >= 2 and spend >= 20):
        return True

    return False


def _should_clone(status, roas, conversions, score, flags):
    """Determine if campaign should be cloned"""
    # Excellent status with strong metrics
    if (status == "excellent" and roas >= 4.0 and
            conversions >= 5 and score >= 85):
        return True

    # Very strong performance
    if (roas >= 5.0 and conversions >= 3 and score >= 80 and
            "scaling_opportunity" in flags):
        return True

    return False


def _needs_creative_refresh(flags, ctr, frequency, conversions, clicks):
    """Determine if creative needs refreshing"""
    creative_flags = ["critically_low_ctr", "low_ctr", "creative_fatigue",
                      "severe_ad_fatigue", "weak_creative"]

    if any(flag in flags for flag in creative_flags):
        return True

    # Low CTR with decent spend
    if ctr < 0.9 and clicks > 50:
        return True

    # High frequency
    if frequency >= 3.0:
        return True

    return False


def _needs_audience_change(flags, cpc, conversion_rate, roas):
    """Determine if audience targeting needs change"""
    audience_flags = ["bad_audience_match", "high_cpc", "low_frequency_high_spend"]

    if any(flag in flags for flag in audience_flags):
        return True

    # High CPC with poor conversion
    if cpc > 1.5 and conversion_rate < 1.5:
        return True

    # Decent ROAS but expensive traffic
    if roas >= 2.0 and cpc > 2.0:
        return True

    return False


def _needs_offer_revision(flags, ctr, conversion_rate, conversions, clicks):
    """Determine if offer/landing page needs revision"""
    offer_flags = ["conversion_dropoff", "low_conversion_rate"]

    if any(flag in flags for flag in offer_flags):
        return True

    # Good traffic, poor conversion
    if ctr >= 1.5 and conversion_rate < 1.0 and clicks > 30:
        return True

    return False


def _needs_budget_optimization(daily_spend, roas, cpc, score):
    """Determine if budget needs optimization"""
    # High spend with mediocre performance
    if daily_spend > 20 and roas < 2.5 and score < 60:
        return True

    # Low spend with good performance (could increase)
    if daily_spend < 15 and roas > 4.0 and cpc < 0.8:
        return True

    return False


def _should_wait(spend, days_running, conversions, impressions):
    """Determine if we should wait for more data"""
    # Very new campaign
    if days_running < 2 and spend < 20:
        return True

    # Low spend, early stage
    if spend < 10 and conversions == 0 and impressions < 1000:
        return True

    return False


# ----------------------------
# Reason Generation Functions
# ----------------------------

def _get_pause_reason(spend, conversions, roas, flags):
    """Generate specific pause reason"""
    if spend >= 30 and conversions == 0:
        return f"🚨 CRITICAL: Spent ${spend:.0f} with zero conversions. Immediate pause required to prevent further loss."
    elif "multiple_issues" in flags:
        return "🚨 Multiple critical issues detected. Pause and restructure campaign completely."
    elif roas < 0.5 and spend >= 20:
        return f"🚨 Extremely poor ROAS ({roas:.2f}) with significant spend. Pause immediately."
    else:
        return "🚨 Campaign is actively losing money. Pause to prevent further losses."


def _get_scale_reason(roas, score, conversions):
    """Generate specific scaling reason"""
    return f"🚀 SCALE: Excellent performance (ROAS: {roas:.1f}x, Score: {score}/100, {conversions} conversions). Increase budget by 25-50%."


def _get_clone_reason(roas, conversions, score):
    """Generate specific cloning reason"""
    return f"🧬 CLONE: Outstanding performance (ROAS: {roas:.1f}x, {conversions} conversions). Duplicate with new audience/creative for horizontal scaling."


def _get_creative_reason(flags, ctr, frequency):
    """Generate specific creative refresh reason"""
    if "severe_ad_fatigue" in flags:
        return f"🎨 URGENT: Severe ad fatigue (frequency: {frequency:.1f}). Refresh creative immediately."
    elif "critically_low_ctr" in flags:
        return f"🎨 URGENT: CTR critically low ({ctr:.2f}%). Complete creative overhaul needed."
    elif ctr < 1.0:
        return f"🎨 Creative underperforming (CTR: {ctr:.2f}%). Test new hooks, visuals, and copy."
    else:
        return "🎨 Creative needs refresh. Test new angles and formats."


def _get_audience_reason(flags, cpc, conversion_rate):
    """Generate specific audience change reason"""
    if "bad_audience_match" in flags:
        return f"🎯 Audience mismatch: High CPC (${cpc:.2f}) with poor conversion rate ({conversion_rate:.1f}%). Refine targeting."
    elif cpc > 2.0:
        return f"🎯 CPC too high (${cpc:.2f}). Test more specific or less competitive audiences."
    else:
        return "🎯 Audience optimization needed. Test narrower targeting or new segments."


def _get_offer_reason(flags, conversion_rate, ctr):
    """Generate specific offer revision reason"""
    if "conversion_dropoff" in flags:
        return f"🛠 Good engagement (CTR: {ctr:.2f}%) but poor conversion ({conversion_rate:.1f}%). Optimize landing page/offer."
    else:
        return f"🛠 Low conversion rate ({conversion_rate:.1f}%). Review pricing, landing page, and value proposition."


def _get_budget_reason(daily_spend, roas, cpc):
    """Generate specific budget optimization reason"""
    if daily_spend > 10 and roas < 2.0:
        return f"💰 High daily spend (${daily_spend:.0f}) with mediocre ROAS. Reduce budget until optimized."
    elif daily_spend < 15 and roas > 3.5:
        return f"💰 Strong performance (ROAS: {roas:.1f}x) with low spend. Consider increasing budget."
    else:
        return "💰 Budget optimization needed for better efficiency."


def _get_wait_reason(spend, days_running, conversions):
    """Generate specific wait reason"""
    if days_running < 2:
        return f"⏳ Campaign too new ({days_running} days). Allow 2-3 days for optimization."
    elif spend < 10:
        return f"⏳ Insufficient spend (${spend:.0f}) for evaluation. Allow more data accumulation."
    else:
        return "⏳ Insufficient data. Monitor for 24-48 hours before making changes."


def _create_decision(decision, reason, metrics, analysis, priority):
    """Create standardized decision output"""
    roas = metrics.get("purchase_roas", 0)
    spend = metrics.get("spend", 0)
    conversions = metrics.get("purchases", 0)

    decision_data = {
        "meta_campaign_id": metrics.get("meta_campaign_id"),
        "decision": decision,
        "reason": reason,
        "priority": priority,
        "confidence": _calculate_confidence(metrics, analysis),
        "metrics_summary": {
            "roas": roas,
            "spend": spend,
            "conversions": conversions,
            "score": analysis.get("score", 0)
        },
        "profitability": "high" if roas >= 3.0 else "medium" if roas >= 1.5 else "low",
        "next_review": _get_next_review_timeframe(decision),
        "expected_outcome": _get_expected_outcome(decision, roas),
        "flags": analysis.get("flags", []),
        "priority_actions": analysis.get("priority_actions", []),
        "recommendations": analysis.get("recommendations", [])
    }
    return decision_data


def _calculate_confidence(metrics, analysis):
    """Calculate confidence level in decision"""
    spend = metrics.get("spend", 0)
    conversions = metrics.get("purchases", 0)
    days_running = metrics.get("days_running", 1)

    if spend >= 30 and conversions >= 3 and days_running >= 3:
        return "high"
    elif spend >= 20 and days_running >= 2:
        return "medium"
    else:
        return "low"


def _get_next_review_timeframe(decision):
    """Get recommended next review timeframe"""
    timeframes = {
        "pause": "24 hours",
        "scale": "48-72 hours",
        "clone": "3-5 days",
        "edit_creative": "24-48 hours",
        "change_audience": "48-72 hours",
        "revise_offer": "24-48 hours",
        "optimize_budget": "48 hours",
        "wait": "24 hours"
    }
    return timeframes.get(decision, "24-48 hours")


def _get_expected_outcome(decision, current_roas):
    """Get expected outcome description"""
    outcomes = {
        "pause": "Stop losses, prepare for restructure",
        "scale": f"Increase profitable returns (target: {current_roas * 1.2:.1f}x ROAS)",
        "clone": "Expand reach while maintaining performance",
        "edit_creative": "Improve CTR and engagement (target: 1.5%+ CTR)",
        "change_audience": "Reduce CPC, improve conversion rate",
        "revise_offer": "Increase conversion rate (target: 2%+)",
        "optimize_budget": "Improve cost efficiency",
        "wait": "Gather sufficient data for informed decisions"
    }
    return outcomes.get(decision, "Improve overall performance")



In [74]:
decision_data = decide_campaign_action(metrics,analysis)

In [75]:
print(decision_data)

{'meta_campaign_id': '123456789012345', 'decision': 'optimize_budget', 'reason': '💰 High daily spend ($33) with mediocre ROAS. Reduce budget until optimized.', 'priority': 'low', 'confidence': 'high', 'metrics_summary': {'roas': 1.6, 'spend': 200.0, 'conversions': 8, 'score': 55}, 'profitability': 'medium', 'next_review': '48 hours', 'expected_outcome': 'Improve cost efficiency', 'flags': ['unprofitable_roas', 'high_spend_low_return'], 'priority_actions': [], 'recommendations': ['🟢 Strong CTR! Your creative is resonating well with the audience.', '🔴 ROAS below breakeven. Optimize targeting, landing page, or pause campaign.', '🟢 CPC is reasonable for your market.', '🟢 Frequency is optimal - good reach without oversaturation.', '🟢 Good conversion rate. Consider scaling.', '🟡 High daily spend with low ROAS. Consider reducing budget until optimized.']}


In [87]:
def modify_campaign_from_decision(decision_data: dict) -> dict:
    """
    Takes the decision output and applies the appropriate modification to the campaign.
    Supports actions like: pause, scale, clone, edit_creative, change_audience, revise_offer, optimize_budget.
    """

    decision = decision_data.get("decision")
    meta_campaign_id = decision_data.get("meta_campaign_id")
    reason = decision_data.get("reason")
    priority = decision_data.get("priority")
    expected_outcome = decision_data.get("expected_outcome")

    # Fetch campaign object
   # try:
        #campaign = agent_models.Campaign.objects.get(meta_campaign_id=meta_campaign_id)
    #except agent_models.Campaign.DoesNotExist:
     #   return {"status": "error", "message": f"Campaign {campaign_id} not found."}

    # Dispatch to action handler
    if decision == "pause":
        return pause_campaign(meta_campaign_id, reason) #✅✅✅
    elif decision == "scale":
        return scale_campaign(meta_campaign_id, decision_data)#✅✅✅
    elif decision == "clone":
        return clone_campaign(meta_campaign_id, decision_data)  #✅✅✅
    elif decision == "edit_creative":
        return refresh_creative(decision_data)  #✅✅✅
    elif decision == "change_audience":
        return update_audience_targeting(decision_data)  #✅✅✅
    elif decision == "revise_offer":
        return revise_offer_logic(decision_data)  #✅✅✅
    elif decision == "optimize_budget":
        return optimize_campaign_budget(decision_data)
    elif decision == "wait":
        return {"status": "waiting", "message": "Campaign under observation. No action taken yet."}

    return {"status": "unhandled", "message": f"Unknown decision: {decision}"}


In [111]:
def optimize_campaign_budget(decision_data: Dict, config: Dict = None):
    """
    Adjusts campaign budget based on ROAS performance with configurable thresholds.
    """
    #from your_app import agent_models  # Ensure this import reflects your actual app
    
    # Default configuration
    default_config = {
        'min_budget': 10.0,
        'max_budget': 500.0,
        'min_change': 1.0,
        'thresholds': {
            'high_roas': 3.0,
            'low_roas': 1.0,
            'scale_up': 1.3,
            'scale_down': 0.7,
            'scale_down_hard': 0.5
        }
    }
    config = {**default_config, **(config or {})}

    campaign = None  # Defensive default

    # 1. Load campaign
    meta_campaign_id = decision_data.get("meta_campaign_id")
    #campaign = agent_models.Campaign.objects.get(meta_campaign_id=meta_campaign_id)
    #adset_id = campaign.meta_adset_id

    #if not adset_id:
        #return {"status": "failed", "error": "Missing meta_adset_id"}

    # 2. Extract metrics
    metrics = decision_data.get("metrics_summary", {})
    roas = float(metrics.get("roas", 0))
    current_budget = 120

    if current_budget <= 0:
        return {"status": "failed", "error": "Invalid current budget"}

    if roas <= 0:
        return {"status": "failed", "error": "Invalid ROAS data"}

    # 3. Determine scaling logic
    thresholds = config['thresholds']
    if roas >= thresholds['high_roas']:
        action = "scale_up"
        factor = thresholds['scale_up']
    elif roas >= thresholds['low_roas']:
        action = "maintain"
        factor = 1.0
    else:
        action = "scale_down"
        factor = thresholds['scale_down_hard'] if roas < 0.5 else thresholds['scale_down']

    # 4. Compute new budget within limits
    new_budget = round(current_budget * factor, 2)
    new_budget = max(config['min_budget'], min(new_budget, config['max_budget']))

    if abs(new_budget - current_budget) < config['min_change']:
        return {"status": "noop", "message": "Budget change below threshold"}

    # 5. Update budget on Meta
    success = False
    for attempt in range(2):
        try:
            response = requests.post(
                f"{BASE_URL}/{adset_id}",
                params={"access_token": ACCESS_TOKEN},
                json={"daily_budget": int(new_budget * 100)},
                timeout=30
            )
            response.raise_for_status()
            success = True
            break
        except requests.RequestException as e:
            if attempt == 1:
                raise e
            time.sleep(1)

    if not success:
        return {"status": "failed", "error": "Meta API update failed"}

    # 6. Update DB
    #campaign.budget = new_budget
    #campaign.save()

    # 7. Log optimization
    #agent_models.OptimizationLog.objects.create(
     #   campaign=campaign,
      #  action="optimize_budget",
       # reason=decision_data.get("reason", f"ROAS-based {action}"),
        #metrics_snapshot=metrics,
        #notes=f"ROAS: {roas:.2f}, Budget: ${current_budget:.2f} → ${new_budget:.2f}"
    #)

    return {
        "status": "success",
        "old_budget": current_budget,
        "new_budget": new_budget,
        "action": action,
        "roas": roas
    }



In [112]:
modify_campaign_from_decision(decision_data)

{'status': 'noop', 'message': 'Budget change below threshold'}